In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys, importlib.util, os
parentPath = Path.cwd().parent
sys.path.insert(0, str(parentPath))  # add project root

In [ ]:
import numpy as np

import mne
from src.preprocessing import raw_data_filter, get_events_and_ids_eegbci, make_epochs
from src.datasets_eegbci import load_eegbci_raws

In [ ]:
filePath = Path(f"{parentPath}/data/raw")
derivedPath = Path(f"{parentPath}/data/derivatives")
derivedPath.mkdir(parents=True, exist_ok=True)

# choose  subjects & MI runs:
SUBJECTS = np.arange(1, 16)  # 1-109
RUNS_MI  = [3,4,5,6,7,8,9,10,11,12,13,14]  # fists L/R; 4/8/12 are IMAGERY, 3/7/11 are EXECUTION

In [ ]:
raws, files = load_eegbci_raws(SUBJECTS, RUNS_MI, cache_dir=filePath)

In [ ]:
for i in range(len(raws)):
    events, anno_map = mne.events_from_annotations(raws[i])
    print(f"File {i+1}/{len(raws)}: Annotations: {list(anno_map.keys())}  Events: {np.unique(events[:,2])}\n ")

In [ ]:
all_epochs = []
summary_rows = []
for raw, fname in zip(raws, files):
    print("Processing:", fname)
    # Basic clean (safe harmonics per sfreq)
    raw_c = raw_data_filter(raw, line_freq=60)  # use 50 if your subject is EU

    # Optional montage
    try:
        raw_c.set_montage('standard_1020')
    except Exception as e:
        print("No montage applied:", e)

    # Events & event_id tailored to the run number
    # events, event_id, inv_map, run = get_events_and_ids_eegbci(raw_c, two_class_only=True)
    # print("Run:", run, "event_id:", event_id)
    events, event_id, inv_map, run = get_events_and_ids_eegbci(raw_c, two_class_only=True)
    # print("Run:", run, "event_id:", event_id)
    # print("Unique event codes in events:", np.unique(events[:,2]))

    # Epochs
    tmin, tmax = -0.2, 0.8
    baseline = (None, 0)
    epochs = make_epochs(raw_c, events, event_id, tmin=tmin, tmax=tmax,
                         reject_ptp_uV=150.0, flat_uV=1.0)

    # quick class counts
    y = epochs.events[:, -1]
    counts = {inv_map[int(k)]: int(v) for k, v in zip(*np.unique(y, return_counts=True))}
    print("Counts:", counts)

    # Save epochs per file
    out = derivedPath / f"epochs_S{Path(fname).name.split('S')[-1].replace('.edf','')}-epo.fif"
    epochs.save(out, overwrite=True)
    print("Saved:", out)

    all_epochs.append(epochs)
    summary_rows.append((fname, run, counts))
len(all_epochs)


In [ ]:
# Choose one epochs object to visualize
epoch_index = 122
epoch = all_epochs[epoch_index]
print(f"Length of epoch: {len(epoch)}")
# epoch.drop_log
epoch.plot_drop_log()
if len(epoch)==0:
    print("⚠️ No epochs to plot (all dropped). Showing drop log instead.")
    fig = epoch.plot_drop_log(show=False)  # always works, even if empty
else:
    psd = epoch.compute_psd(fmin=1, fmax=40, method="welch")
    fig = psd.plot(average=True, show=False)
    # ERP by class name
    labels = list(epoch.event_id.keys())
    for lab in labels:
        epoch[lab].average().plot(spatial_colors=True, titles=f"ERP: {lab}")

In [ ]:
from csv import DictWriter

In [ ]:
EPO_FILES = sorted(derivedPath.glob("*-epo.fif"))
print(len(EPO_FILES), "epoch files found")

rows = []
for f in EPO_FILES:
    ep = mne.read_epochs(f, preload=False, verbose=False)
    y = ep.events[:, -1]
    u, c = np.unique(y, return_counts=True)
    # drop log summary
    dropped = sum(len(x)>0 for x in ep.drop_log)
    kept = len(ep)
    drop_rate = dropped / (dropped + kept) if (dropped+kept)>0 else 0.
    rows.append(dict(
        file=str(f.name),
        sfreq=float(ep.info["sfreq"]),
        n_ch=int(len(ep.ch_names)),
        classes="/".join(ep.event_id.keys()),
        counts=";".join(f"{int(ui)}:{int(ci)}" for ui,ci in zip(u,c)),
        kept=int(kept),
        dropped=int(dropped),
        drop_rate=float(drop_rate),
    ))
len(rows)


In [ ]:
res = Path(f"{parentPath}/results/results.csv")
res.parent.mkdir(parents=True, exist_ok=True)

with res.open("w", newline="") as f:        # <-- 'w' = overwrite
    w = DictWriter(f, fieldnames=list(rows[0].keys()))
    w.writeheader()
    w.writerows(rows)

print(f"Wrote {len(rows)} rows to {res} (overwrote any previous file).")



In [ ]:
FIGS = Path(f"{parentPath}/figs"); FIGS.mkdir(exist_ok=True, parents=True)
for f in EPO_FILES:
    ep = mne.read_epochs(f, preload=True, verbose=False)
    if len(ep) == 0:
        ep.plot_drop_log(show=False).savefig(FIGS / f"droplog_{f.stem}.png", dpi=150); continue

    fig = ep.compute_psd(fmin=1, fmax=40).plot(average=True, show=False)
    fig.savefig(FIGS / f"psd_{f.stem}.png", dpi=150)
